In [ ]:
import tensorflow as tf
import numpy as np

def parse_e2ed(raw_bytes: bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def split_context_name(ctx_name: str):
    if not ctx_name or "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None

def find_record_by_seg_and_idx(tfrecord_file, target_seg_id, target_key_idx, max_scan=None):
    """
    在一个 tfrecord shard 里找某个 segment 的某个 key_idx
    找到则返回 e2e，否则返回 None
    """
    ds = tf.data.TFRecordDataset(
        tfrecord_file,
        compression_type=("GZIP" if tfrecord_file.endswith(".gz") else "")
    )
    for i, r in enumerate(ds):
        if max_scan is not None and i >= max_scan:
            break
        e2e = parse_e2ed(r.numpy())
        ctx = e2e.frame.context.name
        seg_id, key_idx = split_context_name(ctx)
        if seg_id == target_seg_id and key_idx == target_key_idx:
            return e2e
    return None

def _get_repeated(states_obj, field):
    if states_obj is None or (not hasattr(states_obj, field)):
        return None
    return np.array(list(getattr(states_obj, field)), dtype=np.float32)

def show_ego_status_one_record(e2e, head=5):
    """
    轻量展示：只打印你最关心的 ego status（past/future）
    """
    ctx = e2e.frame.context.name
    seg_id, key_idx = split_context_name(ctx)
    print("=== Keyframe ===")
    print("context.name:", ctx)
    print("segment_id:", seg_id)
    print("key_idx:", key_idx)
    print("intent:", int(getattr(e2e, "intent", -1)))

    past = getattr(e2e, "past_states", None)
    fut  = getattr(e2e, "future_states", None)

    # 你目前版本里常见字段（不保证全都有，所以做了容错）
    past_fields = ["pos_x","pos_y","vel_x","vel_y","accel_x","accel_y"]
    fut_fields  = ["pos_x","pos_y","pos_z"]  # 你之前看到 future 有 pos_z

    print("\n=== past_states ===")
    for f in past_fields:
        arr = _get_repeated(past, f)
        if arr is None:
            continue
        print(f"{f}: len={len(arr)} head={arr[:head]} tail={arr[-head:]}")

    print("\n=== future_states ===")
    for f in fut_fields:
        arr = _get_repeated(fut, f)
        if arr is None:
            continue
        print(f"{f}: len={len(arr)} head={arr[:head]} tail={arr[-head:]}")


In [ ]:
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.protos import end_to_end_driving_data_pb2 as wod_e2ed_pb2

FILENAME = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093"
SEG_ID   = "fb0ed944efebd34d756103188d59da85"
KEY_IDX  = 223   # 例子：你之前出现过 20~223

e2e = find_record_by_seg_and_idx(FILENAME, SEG_ID, KEY_IDX, max_scan=None)
print("found:", e2e is not None)

if e2e is not None:
    show_ego_status_one_record(e2e, head=6)


In [ ]:
KEY_IDX  = 20   # 例子：你之前出现过 20~223

e2e = find_record_by_seg_and_idx(FILENAME, SEG_ID, KEY_IDX, max_scan=None)
print("found:", e2e is not None)

if e2e is not None:
    show_ego_status_one_record(e2e, head=6)

新版

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import tensorflow as tf
import numpy as np
from waymo_open_dataset.protos import end_to_end_driving_data_pb2 as wod_e2ed_pb2
import json
import os
from datetime import datetime
import matplotlib
import math
import random

# 1. 解决绘图后端问题
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt


In [ ]:

def parse_e2ed(raw_bytes: bytes):
    m = wod_e2ed_pb2.E2EDFrame()
    m.ParseFromString(raw_bytes)
    return m

def split_context_name(ctx_name: str):
    if not ctx_name or "-" not in ctx_name:
        return ctx_name, None
    base, suf = ctx_name.rsplit("-", 1)
    try:
        return base, int(suf)
    except ValueError:
        return ctx_name, None

def find_record_by_seg_and_idx(tfrecord_file, target_seg_id, target_key_idx, max_scan=None):
    """弹性版本，参考故障恢复文献的隔离原则"""
    try:
        ds = tf.data.TFRecordDataset(
            tfrecord_file,
            compression_type=("GZIP" if tfrecord_file.endswith(".gz") else "")
        )
        
        count = 0
        for i, r in enumerate(ds):
            if max_scan is not None and i >= max_scan:
                print(f"已达到最大扫描限制 {max_scan}")
                break
                
            try:
                e2e = parse_e2ed(r.numpy())
                ctx = e2e.frame.context.name
                seg_id, key_idx = split_context_name(ctx)
                
                if seg_id == target_seg_id and key_idx == target_key_idx:
                    print(f"✅ 在索引 {i} 找到目标记录")
                    return e2e, i  # 返回记录和位置索引
                
                count += 1
                if count % 100 == 0:
                    print(f"已扫描 {count} 条记录...")
                    
            except Exception as e:
                print(f"⚠️ 处理记录 {i} 时出错: {str(e)}")
                continue
                
        print(f"扫描完成，共处理 {count} 条记录")
        return None, None
        
    except Exception as e:
        print(f"❌ 文件读取错误: {str(e)}")
        return None, None

def safe_getattr(obj, attr_name, default=None):
    """安全获取属性，符合故障弹性设计"""
    try:
        return getattr(obj, attr_name)
    except Exception as e:
        return default

def get_field_info(obj, field_name):
    """安全获取字段信息，参考运行时故障预测文献"""
    try:
        value = safe_getattr(obj, field_name)
        if value is None:
            return {"exists": False, "type": "None", "sample": None}
        
        # 处理重复字段
        if hasattr(value, '__len__') and not isinstance(value, (str, bytes)):
            field_type = "repeated"
            length = len(value)
            sample = None
            if length > 0:
                try:
                    # 尝试获取第一个元素的类型
                    if hasattr(value[0], 'DESCRIPTOR'):
                        sample_type = value[0].DESCRIPTOR.name
                        sample = f"<{sample_type}>"
                    else:
                        sample = str(value[0])[:50]
                except Exception as e:
                    sample = f"error getting sample: {str(e)}"
            return {
                "exists": True,
                "type": field_type,
                "length": length,
                "sample": sample
            }
        
        # 处理嵌套消息
        elif hasattr(value, 'DESCRIPTOR'):
            return {
                "exists": True,
                "type": "message",
                "message_type": value.DESCRIPTOR.name,
                "fields": list(value.DESCRIPTOR.fields_by_name.keys())[:5]  # 只取前5个字段
            }
        
        # 基本类型
        else:
            return {
                "exists": True,
                "type": type(value).__name__,
                "value": str(value)[:100]  # 限制长度
            }
            
    except Exception as e:
        return {
            "exists": False,
            "error": str(e),
            "type": "error"
        }

def inspect_e2e_fields(e2e):
    """
    弹性字段检查 - 参考故障恢复文献中的运行时探索原则
    """
    print("="*60)
    print("🔍 弹性结构探索 - WOD-E2E 数据集分析")
    print("="*60)
    
    # 1. 首先检查顶层结构
    print("\n=== 🏗️ 1. 顶层结构分析 ===")
    explore_protobuf(e2e, "  ", max_depth=2)
    
    # 2. 重点检查五个关键字段，使用安全访问
    print("\n" + "="*50)
    print("🎯 2. 五个关键字段详细分析")
    print("="*50)
    
    key_fields = ['frame', 'past_states', 'future_states', 'intent', 'preference_trajectories']
    for field_name in key_fields:
        print(f"\n--- {field_name.upper()} ---")
        field_value = safe_getattr(e2e, field_name)
        if field_value is not None:
            explore_protobuf(field_value, "  ", max_depth=2)
        else:
            print(f"  ⚠️ 字段 '{field_name}' 不存在或为空")
    
    # 3. Intent 字段深度分析
    print("\n" + "="*60)
    print("🧠 3. INTENT 字段深度分析 (参考故障预测文献)")
    print("="*60)
    intent_analysis = analyze_intent_field(e2e)
    
    # 4. Preference Trajectories 深度分析
    print("\n" + "="*60)
    print("🛣️ 4. PREFERENCE TRAJECTORIES 深度分析 (参考故障恢复文献)")
    print("="*60)
    prefs_analysis = analyze_preference_trajectories(e2e)
    
    return intent_analysis, prefs_analysis

def explore_protobuf(obj, prefix="", depth=0, max_depth=3):
    """递归探索protobuf结构，安全版本"""
    if depth > max_depth or obj is None:
        return
    
    if hasattr(obj, 'DESCRIPTOR'):
        fields = list(obj.DESCRIPTOR.fields_by_name.keys())
        print(f"{prefix}🔍 {obj.DESCRIPTOR.name}, 字段数: {len(fields)}")
        
        # 只展示前10个字段避免输出过长
        for field_name in fields[:10]:
            field_info = get_field_info(obj, field_name)
            if field_info["exists"]:
                if field_info["type"] == "message":
                    print(f"{prefix}  📦 '{field_name}': {field_info['message_type']}")
                    if depth < max_depth - 1:  # 限制递归深度
                        nested_obj = safe_getattr(obj, field_name)
                        if nested_obj:
                            explore_protobuf(nested_obj, prefix + "    ", depth + 1, max_depth)
                elif field_info["type"] == "repeated":
                    print(f"{prefix}  📋 '{field_name}': 重复字段, 长度={field_info.get('length', 0)}, 示例={field_info.get('sample', 'N/A')}")
                else:
                    print(f"{prefix}  🔹 '{field_name}': {field_info['type']}, 值={field_info.get('value', 'N/A')}")
            else:
                print(f"{prefix}  ⚠️ '{field_name}': 字段不存在或访问错误")
        
        if len(fields) > 10:
            print(f"{prefix}  ... (还有 {len(fields)-10} 个字段未显示)")
    
    elif hasattr(obj, '__len__') and not isinstance(obj, (str, bytes, np.ndarray)):
        print(f"{prefix}📋 容器类型: {type(obj).__name__}, 长度={len(obj)}")
        if len(obj) > 0:
            try:
                sample = str(obj[0])[:50]
                print(f"{prefix}  示例值: {sample}")
            except:
                pass

def serialize_protobuf_info(obj, max_depth=2, current_depth=0):
    """将protobuf结构转换为可序列化的字典，符合故障隔离原则"""
    if current_depth > max_depth or obj is None:
        return "depth_limit"
    
    result = {}
    
    if hasattr(obj, 'DESCRIPTOR'):
        result["message_type"] = obj.DESCRIPTOR.name
        result["fields"] = {}
        
        # 限制字段数量
        field_names = list(obj.DESCRIPTOR.fields_by_name.keys())[:20]
        
        for field_name in field_names:
            field_info = get_field_info(obj, field_name)
            if field_info["exists"]:
                if field_info["type"] == "message":
                    nested_obj = safe_getattr(obj, field_name)
                    result["fields"][field_name] = {
                        "type": "nested_message",
                        "message_type": field_info["message_type"],
                        "fields_count": len(field_info["fields"]) if "fields" in field_info else 0
                    }
                    # 递归序列化，但限制深度
                    if current_depth < max_depth - 1:
                        nested_info = serialize_protobuf_info(nested_obj, max_depth, current_depth + 1)
                        result["fields"][field_name]["details"] = nested_info
                elif field_info["type"] == "repeated":
                    result["fields"][field_name] = {
                        "type": "repeated",
                        "length": field_info.get("length", 0),
                        "sample": field_info.get("sample", None)
                    }
                else:
                    result["fields"][field_name] = {
                        "type": field_info["type"],
                        "value": field_info.get("value", None)
                    }
            else:
                result["fields"][field_name] = {"type": "error", "error": field_info.get("error", "unknown error")}
    
    return result

def scan_available_indices(tfrecord_file, target_seg_id, max_scan=500):
    """扫描可用索引，参考精确操作文献中的系统性探索"""
    print(f"\n🔍 扫描文件以查找 segment '{target_seg_id}' 的可用索引...")
    try:
        ds = tf.data.TFRecordDataset(
            tfrecord_file,
            compression_type=("GZIP" if tfrecord_file.endswith(".gz") else "")
        )
        
        found_indices = []
        positions = []
        for i, r in enumerate(ds):
            if i >= max_scan:
                break
                
            try:
                e2e = parse_e2ed(r.numpy())
                ctx = e2e.frame.context.name
                seg_id, key_idx = split_context_name(ctx)
                
                if seg_id == target_seg_id and key_idx is not None:
                    found_indices.append(key_idx)
                    positions.append(i)
                    print(f"  ✅ 找到索引: {key_idx} (文件位置: {i})")
                    
            except Exception as e:
                continue
        
        print(f"扫描完成，找到 {len(found_indices)} 个可用索引")
        return found_indices, positions
        
    except Exception as e:
        print(f"❌ 扫描过程中出错: {str(e)}")
        return [], []

def interpret_intent(intent_value):
    """根据文献和常见模式解释 intent 值"""
    # 基于 Failure Prediction at Runtime 文献的意图分类
    intent_mapping = {
        0: "未知/默认意图",
        1: "直行 (Lane Following)",
        2: "左变道 (Left Lane Change)",
        3: "右变道 (Right Lane Change)",
        4: "左转 (Left Turn)",
        5: "右转 (Right Turn)",
        6: "停车 (Stopping)",
        7: "紧急避让 (Emergency Evasion)",
        8: "汇入 (Merging)",
        9: "让行 (Yielding)",
        10: "路口等待 (Intersection Waiting)",
        11: "行人避让 (Pedestrian Yielding)",
        12: "障碍物避让 (Obstacle Avoidance)"
    }
    
    # 基于 Robot Failure Recovery Using VLMs 文献的安全关键意图
    safety_critical_intents = [7, 8, 9, 11, 12]  # 高风险意图
    
    if intent_value in intent_mapping:
        meaning = intent_mapping[intent_value]
        if intent_value in safety_critical_intents:
            meaning += " ⚠️ (高风险意图)"
        return meaning
    else:
        return f"未知意图代码: {intent_value} (需要根据数据集文档确认)"

def predict_failure_risk_from_intent(intent_value):
    """基于意图预测故障风险，参考 Failure Prediction at Runtime 文献"""
    # 基于文献的简化风险模型
    risk_mapping = {
        0: 0.10,  # 未知 - 中等风险
        1: 0.05,  # 直行 - 低风险
        2: 0.15,  # 左变道 - 中等风险
        3: 0.15,  # 右变道 - 中等风险
        4: 0.25,  # 左转 - 较高风险
        5: 0.25,  # 右转 - 较高风险
        6: 0.10,  # 停车 - 低风险
        7: 0.45,  # 紧急避让 - 高风险
        8: 0.35,  # 汇入 - 高风险
        9: 0.30,  # 让行 - 较高风险
        10: 0.20, # 路口等待 - 中等风险
        11: 0.40, # 行人避让 - 高风险
        12: 0.35  # 障碍物避让 - 高风险
    }
    
    base_risk = risk_mapping.get(intent_value, 0.2)  # 默认中等风险
    
    # 添加随机扰动模拟不确定性，参考 Generative Robot Policies 文献
    uncertainty = random.uniform(-0.05, 0.05)
    final_risk = max(0.0, min(1.0, base_risk + uncertainty))
    
    return final_risk

def analyze_intent_field(e2e):
    """专门分析 intent 字段，参考 Failure Prediction at Runtime 文献"""
    intent = safe_getattr(e2e, 'intent')
    
    if intent is None:
        print("  ⚠️ Intent 字段不存在")
        return None
    
    print(f"  🔍 Intent 类型: {type(intent).__name__}")
    print(f"  💡 Intent 值: {intent}")
    
    # 尝试理解 intent 的含义
    intent_meaning = interpret_intent(intent)
    print(f"  📖 Intent 语义: {intent_meaning}")
    
    # 故障预测相关分析
    failure_risk = predict_failure_risk_from_intent(intent)
    print(f"  ⚠️ 故障风险预测: {failure_risk:.2f} (基于意图分析)")
    
    # 可视化建议
    print(f"  💡 可视化建议: 可创建意图分布直方图，分析不同意图下的故障率")
    
    return {
        "value": int(intent),
        "semantic_meaning": intent_meaning,
        "failure_risk": failure_risk
    }

def analyze_single_trajectory(traj, prefix=""):
    """分析单个轨迹"""
    if traj is None:
        print(f"{prefix}⚠️ 轨迹为空")
        return {"error": "轨迹为空"}
    
    analysis = {
        "fields": {},
        "length": 0,
        "safety_score": 0.0
    }
    
    # 检查轨迹字段
    fields_to_check = ['pos_x', 'pos_y', 'pos_z', 'yaw', 'speed', 'acceleration', 'preference_score']
    
    for field in fields_to_check:
        field_analysis = {"exists": False}
        
        if hasattr(traj, field):
            field_analysis["exists"] = True
            try:
                value = getattr(traj, field)
                if hasattr(value, '__len__'):  # 是列表/数组
                    field_analysis["type"] = "repeated"
                    field_analysis["length"] = len(value)
                    if len(value) > 0:
                        try:
                            field_analysis["sample"] = list(value)[:3]
                        except:
                            field_analysis["sample"] = "无法转换为列表"
                else:  # 单个值
                    field_analysis["type"] = "scalar"
                    field_analysis["value"] = float(value) if isinstance(value, (int, float)) else str(value)
            except Exception as e:
                field_analysis["error"] = str(e)
        
        analysis["fields"][field] = field_analysis
        
        # 打印结果
        if field_analysis["exists"]:
            if "type" in field_analysis and field_analysis["type"] == "repeated":
                print(f"{prefix}✅ '{field}': 长度={field_analysis.get('length', 0)}")
                if "sample" in field_analysis:
                    print(f"{prefix}   示例: {field_analysis['sample']}")
            elif "type" in field_analysis and field_analysis["type"] == "scalar":
                print(f"{prefix}✅ '{field}': {field_analysis.get('value', 'N/A')}")
            else:
                print(f"{prefix}✅ '{field}': 存在但类型未知")
        else:
            print(f"{prefix}❌ '{field}': 字段不存在")
    
    # 计算轨迹长度（基于位置数据）
    if hasattr(traj, 'pos_x') and hasattr(traj, 'pos_y'):
        try:
            xs = list(traj.pos_x)
            ys = list(traj.pos_y)
            analysis["length"] = len(xs)
        except:
            pass
    
    # 计算安全分数
    analysis["safety_score"] = calculate_trajectory_safety(traj)
    print(f"{prefix}🛡️ 安全分数: {analysis['safety_score']:.2f}/1.0")
    
    return analysis

def analyze_preference_trajectories(e2e):
    """专门分析 preference_trajectories 字段，参考 Robot Failure Recovery 文献"""
    prefs = safe_getattr(e2e, 'preference_trajectories')
    
    if prefs is None:
        print("  ⚠️ Preference trajectories 字段不存在")
        return None
    
    print("  🔍 Preference trajectories 结构分析:")
    explore_protobuf(prefs, "    ", max_depth=2)
    
    # 检查轨迹数量
    trajectories = safe_getattr(prefs, 'trajectories')
    if trajectories is None or not hasattr(trajectories, '__len__'):
        print("  ⚠️ 无法访问轨迹列表")
        return None
    
    num_trajectories = len(trajectories)
    print(f"  📊 轨迹数量: {num_trajectories}")
    
    trajectory_analyses = []
    
    if num_trajectories > 0:
        print(f"  🎯 首选轨迹分析 (索引 0):")
        primary_analysis = analyze_single_trajectory(trajectories[0], "      ")
        trajectory_analyses.append(primary_analysis)
        
        # 如果有多个轨迹，分析备选轨迹
        if num_trajectories > 1:
            print(f"  🔄 备选轨迹分析 (索引 1-{min(num_trajectories-1, 3)}):")
            for i in range(1, min(num_trajectories, 4)):  # 只分析前3个备选
                print(f"    🛣️ 轨迹 {i}:")
                backup_analysis = analyze_single_trajectory(trajectories[i], "        ")
                trajectory_analyses.append(backup_analysis)
    
    # 故障恢复分析
    recovery_analysis = analyze_trajectories_for_recovery(trajectories)
    print("\n  🛡️ 故障恢复能力分析 (参考 Vision-Language Models 文献):")
    print(f"    {recovery_analysis}")
    
    # 生成建议
    suggestions = generate_recovery_suggestions(trajectories)
    
    return {
        "num_trajectories": num_trajectories,
        "trajectories_analysis": trajectory_analyses,
        "recovery_analysis": recovery_analysis,
        "suggestions": suggestions
    }

def calculate_trajectory_safety(traj):
    """计算轨迹安全性，参考 Precise and Dexterous Manipulation 文献"""
    # 基础安全分数
    safety_score = 1.0
    
    # 位置平滑性（曲率）
    if hasattr(traj, 'pos_x') and hasattr(traj, 'pos_y'):
        try:
            xs = list(traj.pos_x)
            ys = list(traj.pos_y)
            
            if len(xs) > 2:
                # 计算曲率
                curvatures = []
                for i in range(1, len(xs)-1):
                    x1, y1 = xs[i-1], ys[i-1]
                    x2, y2 = xs[i], ys[i]  
                    x3, y3 = xs[i+1], ys[i+1]
                    
                    # 向量
                    v1 = (x2-x1, y2-y1)
                    v2 = (x3-x2, y3-y2)
                    
                    # 曲率近似
                    cross = v1[0]*v2[1] - v1[1]*v2[0]
                    mag1 = math.sqrt(v1[0]**2 + v1[1]**2)
                    mag2 = math.sqrt(v2[0]**2 + v2[1]**2)
                    
                    if mag1 > 0.1 and mag2 > 0.1:  # 避免除零
                        curvature = abs(cross) / (mag1 * mag2)
                        curvatures.append(curvature)
                
                if curvatures:
                    max_curvature = max(curvatures)
                    avg_curvature = sum(curvatures) / len(curvatures)
                    # 高曲率降低安全性
                    safety_score *= max(0.1, 1.0 - max_curvature * 15)
                    safety_score *= max(0.1, 1.0 - avg_curvature * 10)
        except Exception as e:
            pass
    
    # 速度平滑性
    if hasattr(traj, 'speed'):
        try:
            speeds = list(traj.speed)
            if len(speeds) > 1:
                # 计算速度变化率（加速度）
                accels = [abs(speeds[i+1] - speeds[i]) for i in range(len(speeds)-1)]
                max_accel = max(accels) if accels else 0
                # 高加速度降低安全性
                safety_score *= max(0.2, 1.0 - max_accel * 0.5)
        except:
            pass
    
    return max(0.0, min(1.0, safety_score))

def calculate_trajectory_diversity(trajectories):
    """计算轨迹多样性，参考 Failure Resilience 文献"""
    if len(trajectories) < 2:
        return 0.0
    
    # 计算终点位置的分散程度
    endpoints = []
    for traj in trajectories:
        if hasattr(traj, 'pos_x') and hasattr(traj, 'pos_y'):
            try:
                xs = list(traj.pos_x)
                ys = list(traj.pos_y)
                if xs and ys and len(xs) > 0 and len(ys) > 0:
                    endpoints.append((xs[-1], ys[-1]))
            except:
                continue
    
    if len(endpoints) < 2:
        return 0.0
    
    # 计算平均成对距离
    total_dist = 0
    count = 0
    for i in range(len(endpoints)):
        for j in range(i+1, len(endpoints)):
            dx = endpoints[i][0] - endpoints[j][0]
            dy = endpoints[i][1] - endpoints[j][1]
            dist = math.sqrt(dx*dx + dy*dy)
            total_dist += dist
            count += 1
    
    avg_dist = total_dist / count if count > 0 else 0
    # 归一化到0-1范围 (假设100米为最大有意义距离)
    return min(1.0, avg_dist / 100.0)

def analyze_trajectories_for_recovery(trajectories):
    """分析轨迹集合的故障恢复能力"""
    if not trajectories:
        return "无轨迹可用于分析"
    
    # 参考 Robot Failure Recovery Using VLMs 文献
    diversity_score = calculate_trajectory_diversity(trajectories)
    safety_scores = [calculate_trajectory_safety(traj) for traj in trajectories]
    
    analysis = []
    analysis.append(f"轨迹多样性得分: {diversity_score:.2f}/1.0")
    analysis.append(f"安全分数范围: [{min(safety_scores):.2f}, {max(safety_scores):.2f}]")
    analysis.append(f"首选轨迹安全分: {safety_scores[0]:.2f}")
    
    if len(trajectories) > 1:
        backup_safety = max(safety_scores[1:])  # 最佳备选
        analysis.append(f"最佳备选安全分: {backup_safety:.2f}")
        if backup_safety > safety_scores[0] * 0.8:  # 备选至少有首选80%的安全性
            analysis.append("✅ 有高质量备选轨迹，故障恢复能力强")
        else:
            analysis.append("⚠️ 备选轨迹质量较低，故障恢复能力有限")
    
    return "\n    ".join(analysis)

def generate_recovery_suggestions(trajectories):
    """生成故障恢复建议，参考 Vision-Language Models 文献"""
    print("\n  💡 生成式故障恢复建议:")
    
    num_trajectories = len(trajectories)
    if num_trajectories == 0:
        print("    ❌ 无可用轨迹，建议请求远程操作员干预")
        return ["无可用轨迹"]
    
    # 首选轨迹分析
    primary_traj = trajectories[0]
    primary_safety = calculate_trajectory_safety(primary_traj)
    
    suggestions = []
    
    if primary_safety < 0.3:
        suggestions.append("🚨 首选轨迹安全性极低，建议立即切换到备选轨迹")
    elif primary_safety < 0.6:
        suggestions.append("⚠️ 首选轨迹安全性中等，建议监控并准备切换")
    else:
        suggestions.append("✅ 首选轨迹安全性高，可继续执行")
    
    # 备选轨迹分析
    if num_trajectories > 1:
        backup_safeties = [calculate_trajectory_safety(traj) for traj in trajectories[1:]]
        best_backup = max(backup_safeties)
        
        if best_backup > 0.7:
            suggestions.append(f"🛡️ 有高质量备选轨迹 (安全分: {best_backup:.2f})，故障恢复能力强")
        elif best_backup > 0.4:
            suggestions.append(f"🔄 备选轨迹质量一般 (安全分: {best_backup:.2f})，需要谨慎切换")
        else:
            suggestions.append(f"❗ 备选轨迹质量低 (最高安全分: {best_backup:.2f})，建议请求人类干预")
        
        # 轨迹多样性
        diversity = calculate_trajectory_diversity(trajectories)
        if diversity > 0.6:
            suggestions.append("🌈 轨迹多样性高，系统能适应多种场景变化")
        elif diversity > 0.3:
            suggestions.append("🟡 轨迹多样性中等，覆盖部分异常情况")
        else:
            suggestions.append("🔴 轨迹多样性低，系统弹性有限，建议增加轨迹生成策略")
    else:
        suggestions.append("🚫 无备选轨迹，单点故障风险高，建议增加轨迹多样性")
    
    # 生成人类可读建议
    for i, suggestion in enumerate(suggestions, 1):
        print(f"    {i}. {suggestion}")
    
    # 结合文献的高级建议
    print("\n  📚 基于最新研究的高级建议:")
    print("    • 参考 'Robot Failure Recovery Using VLMs'：集成视觉-语言模型进行实时轨迹评估")
    print("    • 参考 'Failure Prediction at Runtime'：在轨迹执行前预测故障概率")
    print("    • 参考 'Precise and Dexterous Manipulation'：使用人类反馈优化轨迹生成")
    
    return suggestions

def save_analysis_results(SEG_ID, KEY_IDX, e2e, file_position, intent_analysis=None, prefs_analysis=None):
    """安全保存分析结果，参考故障恢复文献的持久化原则"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "e2e_analysis_results"
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. 保存结构信息（可序列化版本）
    structure_info = {
        "segment_id": SEG_ID,
        "key_idx": KEY_IDX,
        "file_position": file_position,
        "timestamp": timestamp,
        "top_level_fields": list(e2e.DESCRIPTOR.fields_by_name.keys()),
        "detailed_structure": serialize_protobuf_info(e2e, max_depth=2)
    }
    
    # 添加intent和preference分析
    if intent_analysis is not None:
        structure_info["intent_analysis"] = intent_analysis
    
    if prefs_analysis is not None:
        structure_info["preference_trajectories_analysis"] = prefs_analysis
    
    json_file = os.path.join(output_dir, f"structure_{SEG_ID}_{KEY_IDX}_{timestamp}.json")
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(structure_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 结构信息已保存到: {json_file}")
    
    # 2. 保存关键数据摘要
    summary = {
        "segment_id": SEG_ID,
        "key_idx": KEY_IDX,
        "file_position": file_position,
        "context_name": safe_getattr(e2e.frame, 'context', {}).name if hasattr(e2e, 'frame') else "N/A",
        "timestamp_micros": safe_getattr(e2e.frame, 'timestamp_micros', "N/A") if hasattr(e2e, 'frame') else "N/A"
    }
    
    # 添加 past_states 信息
    if hasattr(e2e, 'past_states'):
        past = e2e.past_states
        summary["past_states"] = {
            "available_fields": [f for f in ['pos_x', 'pos_y', 'vel_x', 'vel_y'] if hasattr(past, f)],
            "sequence_lengths": {}
        }
        for field in ['pos_x', 'pos_y', 'vel_x', 'vel_y']:
            if hasattr(past, field):
                try:
                    arr = list(getattr(past, field))
                    summary["past_states"]["sequence_lengths"][field] = len(arr)
                    if len(arr) > 0:
                        summary["past_states"][f"{field}_sample"] = arr[:3]
                except:
                    pass
    
    # 添加 future_states 信息
    if hasattr(e2e, 'future_states'):
        future = e2e.future_states
        summary["future_states"] = {
            "available_fields": [f for f in ['pos_x', 'pos_y', 'pos_z'] if hasattr(future, f)],
            "sequence_lengths": {}
        }
        for field in ['pos_x', 'pos_y', 'pos_z']:
            if hasattr(future, field):
                try:
                    arr = list(getattr(future, field))
                    summary["future_states"]["sequence_lengths"][field] = len(arr)
                    if len(arr) > 0:
                        summary["future_states"][f"{field}_sample"] = arr[:3]
                except:
                    pass
    
    # 添加intent信息
    if intent_analysis is not None:
        summary["intent"] = intent_analysis
    
    # 添加preference trajectories摘要
    if prefs_analysis is not None:
        summary["preference_trajectories"] = {
            "num_trajectories": prefs_analysis["num_trajectories"],
            "avg_safety_score": sum(t["safety_score"] for t in prefs_analysis["trajectories_analysis"] if "safety_score" in t) / len(prefs_analysis["trajectories_analysis"]) if prefs_analysis["trajectories_analysis"] else 0,
            "recovery_capability": "高" if prefs_analysis["recovery_analysis"] and "✅" in prefs_analysis["recovery_analysis"] else "中/低"
        }
    
    summary_file = os.path.join(output_dir, f"summary_{SEG_ID}_{KEY_IDX}_{timestamp}.json")
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)
    print(f"✅ 数据摘要已保存到: {summary_file}")
    
    # 3. 生成人类可读的报告
    report = f"""
WOD-E2E 数据集分析报告
=======================
生成时间: {timestamp}
Segment ID: {SEG_ID}
Key Index: {KEY_IDX}
文件位置: {file_position}

顶层字段:
{', '.join(e2e.DESCRIPTOR.fields_by_name.keys())}

关键发现:
"""
    
    if hasattr(e2e, 'past_states'):
        past = e2e.past_states
        report += f"\n历史状态字段:\n"
        for field in ['pos_x', 'pos_y', 'vel_x', 'vel_y']:
            if hasattr(past, field):
                try:
                    arr = list(getattr(past, field))
                    report += f"  - {field}: 长度={len(arr)}, 示例={arr[:3]}\n"
                except:
                    report += f"  - {field}: 存在但访问失败\n"
    
    # 添加intent分析
    if intent_analysis is not None:
        report += f"\n意图分析:\n"
        report += f"  - 意图值: {intent_analysis['value']}\n"
        report += f"  - 语义含义: {intent_analysis['semantic_meaning']}\n"
        report += f"  - 故障风险: {intent_analysis['failure_risk']:.2f}\n"
    
    # 添加preference轨迹分析
    if prefs_analysis is not None:
        report += f"\n偏好轨迹分析:\n"
        report += f"  - 轨迹数量: {prefs_analysis['num_trajectories']}\n"
        report += f"  - 故障恢复能力: \n{prefs_analysis['recovery_analysis']}\n"
        report += "\n  - 恢复建议:\n"
        for i, suggestion in enumerate(prefs_analysis['suggestions'], 1):
            report += f"    {i}. {suggestion}\n"
    
    report_file = os.path.join(output_dir, f"report_{SEG_ID}_{KEY_IDX}_{timestamp}.txt")
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report)
    print(f"✅ 人类可读报告已保存到: {report_file}")
    
    return output_dir




In [ ]:
# 主执行流程 - 完全弹性版本
FILENAME = r"/mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093"
SEG_ID   = "fb0ed944efebd34d756103188d59da85"

print("🚀 启动 WOD-E2E 数据集弹性分析工具")
print(f"📁 文件: {FILENAME}")
print(f"🎯 目标 Segment ID: {SEG_ID}")

# 第一步：扫描可用索引
print("\n" + "="*80)
print("1️⃣ 第一步：弹性扫描可用索引（参考故障恢复文献）")
print("="*80)
available_indices, positions = scan_available_indices(FILENAME, SEG_ID, max_scan=300)

if not available_indices:
    print("❌ 未找到任何可用索引。请尝试:")
    print("1. 检查 segment ID 是否正确")
    print("2. 扩大扫描范围 (max_scan)")
    print("3. 检查文件路径和权限")
    exit(1)

print(f"\n✅ 找到 {len(available_indices)} 个可用索引: {available_indices}")
KEY_IDX = available_indices[0]
file_position = positions[0]
print(f"🎯 选择第一个索引: {KEY_IDX} (文件位置: {file_position})")

# 第二步：查找记录
print("\n" + "="*80)
print(f"2️⃣ 第二步：查找 segment '{SEG_ID}' 索引 {KEY_IDX} 的记录")
print("="*80)
e2e, actual_position = find_record_by_seg_and_idx(FILENAME, SEG_ID, KEY_IDX, max_scan=500)

if e2e is not None:
    print(f"\n✅ 成功找到记录! (实际文件位置: {actual_position})")
    
    # 第三步：结构分析
    print("\n" + "="*80)
    print("3️⃣ 第三步：安全结构分析（避免不可序列化对象）")
    print("="*80)
    intent_analysis, prefs_analysis = inspect_e2e_fields(e2e)
    
    # 第四步：安全保存结果
    print("\n" + "="*80)
    print("4️⃣ 第四步：弹性保存分析结果（参考故障预测文献）")
    print("="*80)
    output_dir = save_analysis_results(SEG_ID, KEY_IDX, e2e, actual_position, intent_analysis, prefs_analysis)
    
    print(f"\n🎉 分析完成！所有结果已保存到: {output_dir}")
    print("\n💡 后续建议:")
    print("1. 查看生成的 JSON 文件了解详细结构")
    print("2. 检查 report_*.txt 文件获取人类可读摘要")
    print("3. 根据分析结果调整你的端到端驾驶模型输入")
    
    # 第五步：生成轨迹可视化
    try:
        if hasattr(e2e, 'past_states') and hasattr(e2e.past_states, 'pos_x'):
            past_x = list(e2e.past_states.pos_x)
            past_y = list(e2e.past_states.pos_y)
            
            # 首选轨迹
            if hasattr(e2e, 'future_states') and hasattr(e2e.future_states, 'pos_x'):
                future_x = list(e2e.future_states.pos_x)
                future_y = list(e2e.future_states.pos_y)
                
                # 创建轨迹图
                plt.figure(figsize=(12, 10))
                
                # 历史轨迹
                plt.plot(past_x, past_y, 'b-', linewidth=2, label='历史轨迹', alpha=0.8)
                plt.scatter(past_x[-1], past_y[-1], c='blue', s=100, marker='o', label='当前位置')
                
                # 未来轨迹
                plt.plot(future_x, future_y, 'r-', linewidth=2, label='首选未来轨迹', alpha=0.8)
                plt.scatter(future_x[-1], future_y[-1], c='red', s=100, marker='x', label='目标位置')
                
                # 分析 preference trajectories
                if hasattr(e2e, 'preference_trajectories') and hasattr(e2e.preference_trajectories, 'trajectories'):
                    prefs = e2e.preference_trajectories
                    trajectories = prefs.trajectories
                    
                    # 只绘制前3个备选轨迹
                    for i in range(1, min(len(trajectories), 4)):
                        try:
                            traj = trajectories[i]
                            if hasattr(traj, 'pos_x') and hasattr(traj, 'pos_y'):
                                alt_x = list(traj.pos_x)
                                alt_y = list(traj.pos_y)
                                
                                # 计算安全分数用于颜色和透明度
                                safety = calculate_trajectory_safety(traj)
                                color = plt.cm.viridis(safety)  # 使用 colormap 根据安全分数映射颜色
                                alpha = 0.3 + 0.7 * safety  # 安全性越高，越不透明
                                
                                label = f'备选轨迹 {i} (安全分: {safety:.2f})'
                                plt.plot(alt_x, alt_y, color=color, linewidth=1.5, alpha=alpha, label=label)
                        except Exception as e:
                            continue
                
                # 添加意图信息
                if intent_analysis is not None:
                    plt.text(0.05, 0.95, f"当前意图: {intent_analysis['semantic_meaning']}\n" +
                            f"故障风险: {intent_analysis['failure_risk']:.2f}", 
                            transform=plt.gca().transAxes, fontsize=10,
                            bbox=dict(facecolor='yellow', alpha=0.3, edgecolor='none'))
                
                plt.title(f'轨迹规划分析 - Segment: {SEG_ID}, Index: {KEY_IDX}')
                plt.xlabel('X 位置 (m)')
                plt.ylabel('Y 位置 (m)')
                plt.legend(loc='best')
                plt.grid(True)
                plt.axis('equal')
                
                
                viz_file = os.path.join(output_dir, f"trajectory_analysis_{SEG_ID}_{KEY_IDX}.png")
                plt.savefig(viz_file)
                plt.close()
                print(f"📈 轨迹可视化已保存到: {viz_file}")
                
    except Exception as e:
        print(f"⚠️ 生成可视化时出错 (可忽略): {str(e)}")
    
else:
    print("❌ 未能找到记录。弹性恢复建议:")
    print("1. 尝试其他可用索引:", available_indices[1:])
    print("2. 检查 Waymo Open Dataset 版本是否匹配")
    print("3. 验证 protobuf 定义文件是否为最新版本")
    print("4. 根据故障恢复文献，考虑使用备用数据源")
    
    


🚀 启动 WOD-E2E 数据集弹性分析工具
📁 文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093
🎯 目标 Segment ID: fb0ed944efebd34d756103188d59da85

1️⃣ 第一步：弹性扫描可用索引（参考故障恢复文献）

🔍 扫描文件以查找 segment 'fb0ed944efebd34d756103188d59da85' 的可用索引...
  ✅ 找到索引: 223 (文件位置: 78)
扫描完成，找到 1 个可用索引

✅ 找到 1 个可用索引: [223]
🎯 选择第一个索引: 223 (文件位置: 78)

2️⃣ 第二步：查找 segment 'fb0ed944efebd34d756103188d59da85' 索引 223 的记录
✅ 在索引 78 找到目标记录

✅ 成功找到记录! (实际文件位置: 78)

3️⃣ 第三步：安全结构分析（避免不可序列化对象）
🔍 弹性结构探索 - WOD-E2E 数据集分析

=== 🏗️ 1. 顶层结构分析 ===
  🔍 E2EDFrame, 字段数: 5
    📦 'frame': Frame
      🔍 Frame, 字段数: 11
        📦 'context': Context
        🔹 'timestamp_micros': int, 值=0
        📦 'pose': Transform
        📋 'images': 重复字段, 长度=8, 示例=<CameraImage>
        📋 'lasers': 重复字段, 长度=0, 示例=None
        📋 'laser_labels': 重复字段, 长度=0, 示例=None
        📋 'projected_lidar_labels': 重复字段, 长度=0, 示例=None
        📋 'camera_labels': 重复字段, 长度=0, 示例=None
        📋 'no_label_zones': 重复字段, 长度=0, 示例=None
        📋 'map_features': 重复字段, 长度=

/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 20301 (\N{CJK UNIFIED IDEOGRAPH-4F4D}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 32622 (\N{CJK UNIFIED IDEOGRAPH-7F6E}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 36712 (\N{CJK UNIFIED IDEOGRAPH-8F68}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 36857 (\N{CJK UNIFIED IDEOGRAPH-8FF9}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 35268 (\N{CJK UNIFIED IDEOGRAPH-89C4}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 21010 (\N{CJK UNIFIED IDEOGRAPH-5212}) missing from current font.
  plt.savefig(viz_file)
/tmp/ipykernel_27870/566089376.py:115: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from cu